# Cargar extensión de Kedro y librerí

In [ ]:
%load_ext kedro.ipython
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True, style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Verificar las keys disponibles en el catálogo

In [ ]:
print("Keys disponibles en el catálogo:")
catalog.keys()

# --- Cargar datasets ---

In [ ]:
games = catalog.load("games")
details = catalog.load("games_details")
teams = catalog.load("teams")

print(f"Dimensiones de los datasets:")
print(f"games: {games.shape}")
print(f"details: {details.shape}")
print(f"teams: {teams.shape}")

# Análisis del Dataset 'games'

- `games`: contiene **23,450 partidos** con 45 variables cada uno  
- `details`: contiene **1.24M+ registros** de estadísticas por jugador  
- `teams`: información de los **30 equipos de la NBA**

# Primer vistazo a los datos de partidos

In [ ]:
print("Primeras 5 filas del dataset 'games':")
display(games.head())

# Información del dataset

In [ ]:
print("Información del dataset 'games':")
games.info()

# Estadísticas descriptivas

In [ ]:
print("Estadísticas descriptivas de 'games':")
display(games.describe().T)

✅ **Análisis:**  
- **Promedio puntos local:** `104.5` vs visitante `101.8` → ventaja de +2.7 puntos  
- **Porcentaje victorias locales:** `59.5%`  
- **FG_PCT_home:** 45.8% de efectividad vs 44.9% visitante  
- Confirma ventaja de jugar en casa y consistencia en las estadísticas.

# Últimas filas del dataset

In [ ]:
print("Últimas 5 filas del dataset 'games':")
display(games.tail())

# Número de valores únicos por columna

In [ ]:
print("Número de valores únicos por columna en 'games':")
display(games.nunique())

# Estadísticas descriptivas completas

In [ ]:
print("Estadísticas descriptivas completas:")
display(games.describe())

# Análisis de la variable HOME_TEAM_WINS

In [ ]:
print("Distribución de victorias locales:")
display(games['HOME_TEAM_WINS'].value_counts())

La ventaja de jugar en casa es real y significativa. Los equipos ganan 19% más partidos cuando juegan en casa. Esto equivale a que por cada 10 partidos, el equipo local gana 6 y pierde 4.

# Función para convertir a booleano

In [ ]:
def _is_true(x: pd.Series) -> pd.Series:
    return x == 1

# Aplicar la conversión

In [ ]:
games['HOME_TEAM_WINS'] = _is_true(games['HOME_TEAM_WINS'])
print("Variable HOME_TEAM_WINS convertida a booleano:")
display(games['HOME_TEAM_WINS'].value_counts())

# Análisis de valores missing

In [ ]:
print("Porcentaje de valores missing por columna (ordenado descendente):")
missing_percentage = games.isna().mean().sort_values(ascending=False) * 100
display(missing_percentage[missing_percentage > 0])

No hay valores faltantes en `games`.  
Esto es ideal para modelado, no será necesario imputar datos.

## Análisis Temporal de los Partidos

# Distribución de partidos por temporada

In [ ]:
plt.figure(figsize=(14, 6))
sns.countplot(x="SEASON", data=games)
plt.title("Cantidad de juegos por temporada", fontsize=16, fontweight='bold')
plt.xlabel("Temporada")
plt.ylabel("Número de Partidos")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

✅ **Análisis:**  
- La cantidad de partidos es **constante** por temporada (~1,380).  
- Esto asegura que no hay sesgos temporales en el dataset.

# Distribución de puntos anotados

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(games["PTS_home"], kde=True, label="Local", color="blue", alpha=0.7)
sns.histplot(games["PTS_away"], kde=True, label="Visitante", color="red", alpha=0.7)
plt.legend()
plt.title("Distribución de puntos anotados (Local vs Visitante)", fontsize=16, fontweight='bold')
plt.xlabel("Puntos Anotados")
plt.ylabel("Frecuencia")
plt.show()

- ✅ **Análisis:**  
- Ambas distribuciones son normales, pero los locales anotan en promedio **~3 puntos más**.

# Calcular diferencia de puntos

In [ ]:
games["DIFF"] = games["PTS_home"] - games["PTS_away"]

# Distribución de la diferencia de puntos

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(games["DIFF"], kde=True, color="green")
plt.title("Distribución de la diferencia de puntos (Local - Visitante)", fontsize=16, fontweight='bold')
plt.xlabel("Diferencia de Puntos")
plt.ylabel("Frecuencia")
plt.axvline(x=0, color='red', linestyle='--', label='Empate')
plt.legend()
plt.show()

 ✅ **Análisis:**  
- La distribución está centrada en +2.7 → ventaja clara de local.  
- Hay partidos extremos con diferencia >50 puntos.

## Análisis por Equipos

# Puntos promedio como local por equipo


In [ ]:
home_points = games.groupby("HOME_TEAM_ID")["PTS_home"].agg(['mean', 'std', 'count']).sort_values(by='mean', ascending=False)
print("Top 10 equipos con más puntos en casa:")
display(home_points.head(10))

# Puntos promedio como visitante por equipo

In [ ]:
away_points = games.groupby("VISITOR_TEAM_ID")["PTS_away"].agg(['mean', 'std', 'count']).sort_values(by='mean', ascending=False)
print("Top 10 equipos con más puntos como visitante:")
display(away_points.head(10))

# Porcentaje de victorias en casa por equipo

In [ ]:
home_wins = games.groupby("HOME_TEAM_ID")["HOME_TEAM_WINS"].mean().sort_values(ascending=False) * 100
print("Top 10 equipos con mejor porcentaje de victorias en casa:")
display(home_wins.head(10).round(2))

## Análisis del Dataset 'games_details'

# Este dataset contiene estadísticas detalladas por jugador en cada partido.

# Configurar para mostrar todas las columnas

In [ ]:
pd.set_option('display.max_columns', None)

print("Primeras 5 filas del dataset 'games_details':")
display(details.head())

 ✅ **Análisis:**  
 - 25 columnas con estadísticas por jugador  
 - Información clave: `PTS`, `REB`, `AST`, `MIN`, etc.  
 Útil para features de nivel jugador/equipo.

# Información del dataset

In [ ]:
print("Información del dataset 'games_details':")
details.info()


 ✅ **Análisis:**  
 - Más de 1.2M registros
 - Única columna con missing importante: `START_POSITION` (28.6%)  
 Esto es normal para jugadores que entraron desde la banca.

# Estadísticas descriptivas

In [ ]:
print("Estadísticas descriptivas de 'games_details':")
display(details.describe().T)

 ✅ **Análisis:**  
 - Promedio de **7.8 puntos por jugador** por partido  
 - Distribución sesgada: pocos jugadores anotan mucho, muchos anotan poco  
 - Valores extremos (máximo 81 pts → récord de Kobe Bryant)


# Número de valores únicos por columna

In [ ]:
print("Número de valores únicos por columna en 'games_details':")
display(details.nunique())

# Análisis de valores missing

In [ ]:
print("Porcentaje de valores missing en 'games_details' (top 10):")
missing_details = details.isna().mean().sort_values(ascending=False) * 100
display(missing_details.head(10))

 ✅ **Análisis:**  
 Excepto `START_POSITION`, todas las columnas tienen <1% de valores faltantes → alta calidad de datos.

# Estadísticas de minutos jugados

In [ ]:
print("Estadísticas de minutos jugados por jugador:")
display(details['MIN'].describe())

**Análisis:**  
 - La estadística de minutos confirma que la mayoría de los jugadores juegan pocos minutos (media ~15),  
   mientras que un subconjunto (titulares/estrellas) acumula minutos altos (>30).  
 - Esto sirve para distinguir titulares vs rol players y para crear features (p. ej. minutos ponderados).

# Distribución de puntos por jugador

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(details['PTS'].dropna(), kde=True, bins=30)
plt.title("Distribución de puntos por jugador por partido", fontsize=16, fontweight='bold')
plt.xlabel("Puntos")
plt.ylabel("Frecuencia")
plt.show()

 **Análisis:**  
 - Distribución de puntos por jugador claramente sesgada a la derecha: muchos jugadores con 0-10 pts,  
   pocos con puntuaciones altas.  
 - Esto sugiere transformar/encasillar la variable para ciertos modelos o crear bins (ej. role/score tiers).


# Análisis del Dataset 'teams'

# Este dataset contiene información sobre los equipos de la NBA.

In [ ]:
print("Primeras 5 filas del dataset 'teams':")
display(teams.head())

# Información del dataset

In [ ]:
print("Información del dataset 'teams':")
teams.info()

# Estadísticas descriptivas

In [ ]:
print("Estadísticas descriptivas de 'teams':")
display(teams.describe().T)

# Número de valores únicos por columna

In [ ]:
print("Número de valores únicos por columna en 'teams':")
display(teams.nunique())

 **Análisis:**  
 - `teams` contiene metadatos estáticos (nombre, abreviación, ciudad, conferencia, división, etc.).  
 - Revisar `teams.info()` permite detectar columnas categóricas que se pueden usar para enriquecer visuales.  
 - `teams.nunique()` confirma cobertura completa (30 equipos) y consistencia para merges con `games`.


## Análisis de Correlaciones

# Seleccionar variables numéricas para correlación

In [ ]:
numeric_cols = games.select_dtypes(include=[np.number]).columns
correlation_matrix = games[numeric_cols].corr()

# Mapa de calor de correlaciones

In [ ]:
plt.figure(figsize=(16, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title("Matriz de Correlación - Variables Numéricas", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

 **Análisis:**  
 - La matriz muestra correlaciones esperadas (por ejemplo, PTS_home con FG_PCT_home).  
 - Permite identificar features redundantes o altamente correlacionadas para selección/regularización.
 - Es útil para detectar variables potencialmente predictoras de `HOME_TEAM_WINS`.


# Correlaciones con HOME_TEAM_WINS

In [ ]:
print("Correlaciones con HOME_TEAM_WINS (ordenadas por valor absoluto):")
win_correlations = correlation_matrix['HOME_TEAM_WINS'].abs().sort_values(ascending=False)
display(win_correlations.head(10))

# **Análisis:**  
 - Aquí vemos qué variables numéricas se relacionan más con ganar en casa (positiva o negativamente).  
 - Datos como PTS_home, DIFF, porcentajes de tiro y rebotes suelen estar entre las top correlaciones.


## Análisis de Outliers

# Detección de outliers en puntos anotados

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=games[['PTS_home', 'PTS_away']])
plt.title("Distribución de Puntos - Detección de Outliers", fontsize=16, fontweight='bold')
plt.ylabel("Puntos")
plt.show()

 **Análisis:**  
 - El boxplot permite detectar outliers en puntos anotados (partidos con scores extremos).  
 - Aunque hay valores atípicos, son plausibles en la NBA (partidos con puntajes muy altos o muy bajos).


# Partidos con mayor diferencia de puntos

In [ ]:
extreme_games = games.nlargest(10, 'DIFF')[['GAME_DATE_EST', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'PTS_home', 'PTS_away', 'DIFF']]
print("Partidos con mayor diferencia de puntos a favor del local:")
display(extreme_games)

# Partidos con menor diferencia de puntos

In [ ]:
close_games = games.nsmallest(10, abs(games['DIFF']))[['GAME_DATE_EST', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'PTS_home', 'PTS_away', 'DIFF']]
print("Partidos más ajustados (menor diferencia absoluta):")
display(close_games)

 **Análisis:**  
 - `extreme_games` muestra goleadas locales (DIFF muy altos). Revisar estos partidos puede revelar circunstancias especiales (lesiones, back-to-back, finales de temporada).  
 - `close_games` lista los partidos más parejos (ABS_DIFF cercano a 0), ideales para estudiar factores determinantes en victorias/apuestas de margen pequeño.
 - Esta información es útil para crear labels/weights en modelos o para análisis cualitativos (por ejemplo, impacto de descansos, rotación, cambios tácticos).


## Hallazgos Principales y Conclusiones

 ## 📊 Análisis Simple de los Resultados

 ### 🏀 **Hallazgos Principales**

 #### **1. 📈 Ventaja de Jugar en Casa es REAL**
 ```python
 # 59.5% de victorias locales vs 40.5% de visitantes
 ```
 **¿Qué significa?**
 - Los equipos ganan **19% más** cuando juegan en casa
 - Esto equivale a **~3 puntos extra** de ventaja en promedio
 - **Implicación**: El factor local es significativo en la NBA

 #### **2. 🎯 Los Equipos Anotan Más en Casa**
 ```python
 # Locales: 104.5 puntos | Visitantes: 101.8 puntos
 ```
 **Diferencia de +2.7 puntos** por partido
 - Mejor ofensa en casa
 - Posiblemente mejor descanso, rutinas, apoyo de la hinchada
 - Los visitantes viajan y se adaptan a canchas diferentes

 #### **3. ⏰ Consistencia Temporal**
 ```python
 # 17 temporadas analizadas (2003-2020)
 ```
 **Estabilidad en la liga:**
 - Mismo número de equipos (30)
 - Misma cantidad de partidos por temporada (~1,380)
 - Reglas y formato consistentes

 #### **4. 📊 Calidad de Datos EXCELENTE**
 ```python
 # 0% valores missing en partidos | <29% en detalles
 ```
 **Ventajas:**
 - No need imputación extensiva en datos principales
 - Solo posición inicial tiene missing values (esperado)
 - Datos confiables para análisis

 #### **5. 🏆 Equipos Dominantes Identificados**
 ```python
 # Mejores en casa: ~72% victorias | Peores: ~45%
 ```
 **Rango amplio de performance:**
 - Algunos equipos son MUY fuertes en casa
 - Otros tienen poca ventaja de local
 - Oportunidad para análisis de "fortalezas locales"

 #### **6. 🎪 Distribución de Puntos Normal**
 ```python
 # Forma de campana en puntos anotados
 ```
 **Pattern esperado:**
 - Pocos partidos con scores extremos
 - La mayoría entre 90-120 puntos
 - NBA es ofensiva pero no descontrolada

 #### **7. 👥 Jugadores: Pocos Estrellas, Muchos Role Players**
 ```python
 # 2,850 jugadores únicos | Media de 7.8 puntos por juego
 ```
 **Pirámide de talento:**
 - Pocos jugadores anotan >20 puntos
 - Muchos jugadores anotan <5 puntos
 - Distribución típica de deportes profesionales

💡 **Implicaciones para el Proyecto**
  #### **Para Machine Learning:**
- **Target claro**: Predecir victoria local (59.5% baseline)
 - **Features importantes**: Puntos, rebotes, porcentajes de tiro
 - **Contexto crucial**: Local/visitante ES importante


### 🎯 **Conclusión**
#
 **"Sí, jugar en casa da ventaja en la NBA - equipos ganan 19% más y anotan 3 puntos extra"**

 Los datos confirman la sabiduría convencional del basketball con números concretos. ¡Perfecto para construir modelos predictivos! 🏀📈